In [2]:
!pip install sklearn

In [4]:

def evaluate_forward(data, model, name):
    model.eval()
    batch_size = 1
    start_time = time.time()
    y_ans = []
    y_pred = []
    for batch in data.batch_iter(name,batch_size,False):
        gaz_list, batch_word, batch_biword, batch_wordlen, batch_char, batch_charlen, batch_charrecover, batch_pos1, batch_pos2, ins_label, batch_label, mask, scope = batch
        prob = model(gaz_list, batch_word, batch_biword, batch_wordlen, batch_char, batch_charlen, batch_charrecover, batch_pos1, batch_pos2, ins_label, scope)
        
        prob = prob.cpu().data.numpy()
        assert batch_size == len(batch_label)
        for bid in range(batch_size):
            cur_ans = batch_label[bid]
            cur_ans = list(set(cur_ans))
            cur_prob = prob[bid]
            y_ans.append(cur_ans)
            y_pred.append(cur_prob)
    
    return y_ans,y_pred   

In [5]:
# -*- coding: utf-8 -*-
# author: huihui
# date: 2020/4/1 2:59 下午 

import time
import configure
from utils.data_manager import *
from nn.framework import MGLattice_model

public_path = configure.public_path
dataset = os.path.join(public_path, configure.dataset)
test_file = os.path.join(dataset, configure.test_file)
print('test_file={}'.format(test_file))

data = load_data_setting(configure.savedset)
data.generate_instance_with_gaz(test_file, 'test', load_mode='multilab-ins')

model_dir = configure.loadmodel
print('Load model from ', model_dir)
model = MGLattice_model(data)
model.load_state_dict(torch.load(model_dir, map_location='cpu'))
y_ans, y_pred = evaluate_forward(data, model, 'test')

print(y_ans)
print(y_pred)


test_file=data/FinRE/test.txt
Data setting loaded from file:  models/FinRE.pkl.dset
DATA SUMMARY START:
     MAX SENTENCE LENGTH: 86
     Number   normalized: True
     Use          bigram: False
     Word  alphabet size: 3069
     Biword alphabet size: 94785
     Char  alphabet size: 3068
     Gaz   alphabet size: 26430
     Label alphabet size: 44
     Word embedding size: 100
     Biword embedding size: 50
     Char embedding size: 30
     Gaz embedding size: 200
     Norm     word   emb: True
     Norm     biword emb: True
     Norm     gaz    emb: False
     Norm   gaz  dropout: 0.5
     Train instance number: 0
     Dev   instance number: 0
     Test  instance number: 0
     Hyperpara  iteration: 100
     Hyperpara  batch size: 1
     Hyperpara          lr: 0.015
     Hyperpara    lr_decay: 0.05
     Hyperpara     HP_clip: 5.0
     Hyperpara  hidden_dim: 200
     Hyperpara     dropout: 0.5
     Hyperpara  lstm_layer: 1
     Hyperpara      bilstm: False
     Hyperpara         GPU:

RuntimeError: set_storage is not allowed on a Tensor created from .data or .detach().
If your intent is to change the metadata of a Tensor (such as sizes / strides / storage / storage_offset)
without autograd tracking the change, remove the .data / .detach() call and wrap the change in a `with torch.no_grad():` block.
For example, change:
    x.data.set_(y)
to:
    with torch.no_grad():
        x.set_(y)